In [1]:
from netgen.csg import *
from ngsolve import *
from ngsolve.webgui import Draw
# import netgen.gui

import numpy as np

In [2]:
# in SI units
a = 0.150 # width of hole in coil
l = 0.294 # length of aluminium plate
r = 0.025  # width of coil

# hights of coil
hc0 = 0.049 
hc1 = 0.149 

ha = 0.019  # hight of aluminium plate

# x-y coordinates of vertices for coil 
'''(Eckpunkte des die Spule umschließenden Quaders, wie sag ich das auf Englisch?)'''
cv = [(l-a-2*r,0), (l,0), (l,a+2*r), (l-a-2*r,a+2*r)]

# x-y coordinates of centers of circles for corners
cc = [(l-a, 2*r), (l-2*r, 2*r), (l-2*r, a), (l-a, a)]

# x-y coordinates of vertices for hole
hole_vertices = [(0.018, 0.018), (0.126, 0.126)]

d = 0.050 # extra space between geometry and enclosing box

sor = 1 # sense of rotation

In [3]:
geo = CSGeometry()

''' warum brauch ich '-1e-3' damit es funktioniert? '''
p = cv.copy()
p[0] = (p[0][0], -1e-3)

maxh_corners = 0.020
# corners
corners = []

'''
########## was ich gerne haette ##########
for i in range(4):
    corners += [(Cylinder(Pnt(cc[i]+(0,)), Pnt(cc[i]+(1,)), 2*r) - Cylinder(Pnt(cc[i]+(0,)), Pnt(cc[i]+(1,)), r)) * \
       OrthoBrick(Pnt(p[i]+(hc0,)),Pnt(cc[i]+(hc1,)))]
####################
'''

'''########## was ich stattdessen mache ##########'''
corners += [(Cylinder(Pnt(cc[0]+(0,)), Pnt(cc[0]+(1,)), 2*r) - Cylinder(Pnt(cc[0]+(0,)), Pnt(cc[0]+(1,)), r)) * \
             OrthoBrick(Pnt(p[0]+(hc0,)),Pnt(cc[0]+(hc1,))).mat('corner0').maxh(maxh_corners)]
corners += [(Cylinder(Pnt(cc[1]+(0,)), Pnt(cc[1]+(1,)), 2*r) - Cylinder(Pnt(cc[1]+(0,)), Pnt(cc[1]+(1,)), r)) * \
             OrthoBrick(Pnt(p[1]+(hc0,))-Vec(2*r, 0,0),Pnt(cc[1]+(hc1,))+Vec(2*r,0,0)).mat('corner1').maxh(maxh_corners)]
corners += [(Cylinder(Pnt(cc[2]+(0,)), Pnt(cc[2]+(1,)), 2*r) - Cylinder(Pnt(cc[2]+(0,)), Pnt(cc[2]+(1,)), r)) * \
             OrthoBrick(Pnt(cc[2]+(hc0,)),Pnt(p[2]+(hc1,))).mat('corner2').maxh(maxh_corners)]
corners += [(Cylinder(Pnt(cc[3]+(0,)), Pnt(cc[3]+(1,)), 2*r) - Cylinder(Pnt(cc[3]+(0,)), Pnt(cc[3]+(1,)), r)) * \
             OrthoBrick(Pnt(p[3]+(hc0,))-Vec(0,2*r,0), Pnt(cc[3]+(hc1,))+Vec(0,2*r,0)).mat('corner3').maxh(maxh_corners)]
'''####################'''


# bricks
b0 = [(l-a,0), (l-r,2*r), (l-a,a+r), (l-a-2*r,2*r)]
b1 = [(l-2*r,r),(l,a), (l-2*r,a+2*r), (l-a-r,a)]
bricks = []
for i in range(4):
    bricks += [OrthoBrick(Pnt(b0[i]+(hc0,)), Pnt(b1[i]+(hc1,))).mat('brick{}'.format(i))]

plate = (OrthoBrick(Pnt(0,0,0), Pnt(l,l,ha)) -
            OrthoBrick(Pnt(hole_vertices[0]+(0,)), Pnt(hole_vertices[1]+(ha,)))).mat('plate')


box = OrthoBrick(Pnt(0-d, 0-d, 0-d), Pnt(p[2][0]+d, l+d, hc1+d)).bc("outer")
air = (box - sum(bricks) - sum(corners) - plate).mat('air')
    

geo.Add(air)
geo.Add(plate)
for i in range(4):
    geo.Add(corners[i])
    geo.Add(bricks[i])
# generate mesh
mesh = Mesh(geo.GenerateMesh(maxh=0.5))
mesh.Curve(6)


# generate normalized impressed current vectors
n = lambda center: CoefficientFunction(sqrt((x-center[0])**2+(y-center[1])**2))
sd = {
    'corner0': (-sor*(y-cc[0][1])/n(cc[0]),sor*(x-cc[0][0])/n(cc[0]),0) ,
    'corner1': (-sor*(y-cc[1][1])/n(cc[1]),sor*(x-cc[1][0])/n(cc[1]),0) , 
    'corner2': (-sor*(y-cc[2][1])/n(cc[2]),sor*(x-cc[2][0])/n(cc[2]),0) ,
    'corner3': (-sor*(y-cc[3][1])/n(cc[3]),sor*(x-cc[3][0])/n(cc[3]),0) ,
    'brick0': (sor*1,0,0) ,
    'brick1': (0,sor*1,0) ,
    'brick2': (-sor*1,0,0) ,
    'brick3': (0,-sor*1,0)
}
coil_mat = ['corner0','corner1', 'corner2','corner3','brick0','brick1','brick2','brick3']
source_norm = CoefficientFunction([sd[mat] if mat in sd.keys() else (0,0,0) for mat in mesh.GetMaterials()])

Draw (source_norm, mesh, draw_surf=False)

NGSWebGuiWidget(value={'ngsolve_version': '6.2.2008-17-g2ebed44be', 'mesh_dim': 3, 'order2d': 2, 'order3d': 2,…

In [4]:
SHOW_POTENTIAL = False
if SHOW_POTENTIAL: 
    %matplotlib
    from matplotlib import pyplot as plt
    from mpl_toolkits import mplot3d
    
import numpy as np

def T0z(x,y,z, sor=-1):
    
    # check if z is in area of coil
    if (z>hc1 or z<hc0): return(0)
    
    # check if x is in area 1 or 2
    if (x >= cv[0][0] and x<cc[0][0]):
        
        # check if y is in area D or E
        if (y >= cv[0][1] and y<cc[0][1]):
            
            dist = lambda p: sqrt((p[0]-cc[0][0])**2+(p[1]-cc[0][1])**2)
            if dist((x,y)) > 2*r: return (0)
            elif dist((x,y)) < r: return (sor)
            else: return (sor*(2*r-dist((x,y)))/r)
        
        # check if y is in area C
        elif (y >= cc[0][1] and y < cc[3][1]):
            
            if (x <= cv[0][0]+r): 
                return(sor*(x-cv[0][0])/r)
            else: return (sor)
        
        # check if y is in area A or B
        elif (y >= cc[3][1] and y <= cv[3][1]):
            
            dist = lambda p: sqrt((p[0]-cc[3][0])**2+(p[1]-cc[3][1])**2)
            if dist((x,y)) > 2*r: return (0)
            elif dist((x,y)) < r: return (sor)
            else: return (sor*(2*r-dist((x,y)))/r)
        
     
    # check if x is in area 3
    elif (x>= cc[0][0] and x<cc[1][0]):
        
        # check if y is in area E
        if (y>cv[0][1] and y < cv[0][1]+r):
            return(sor*y/r)
        
        # check if y is in area B to D
        elif (y >= cv[0][1]+r and y < cv[3][1]-r): 
            return (sor)
        
        # check if y is in area A
        elif (y >= cv[3][1]-r and y < cv[3][1]):
            return(-sor*(y-cv[3][1])/r)

    # check if x is in area 4 or 5
    elif (x >= cc[1][0] and x <= cv[1][0]):
        
        # check if y is in area D or E
        if (y>=cv[0][1] and y<cc[0][1]):
            
            dist = lambda p: sqrt((p[0]-cc[1][0])**2+(p[1]-cc[1][1])**2)
            if dist((x,y)) > 2*r: return (0)
            elif dist((x,y)) < r: return (sor)
            else: return (sor*(2*r-dist((x,y)))/r)
        
        # check if y is in area C
        elif (y >= cc[0][1] and y < cc[3][1]):
            
            if ( x >= cv[1][0]-r): 
                return(-sor*(x-cv[1][0])/r)
            
            else: return (sor)
        
        # check if y is in area A or B
        elif (y >= cc[3][1] and y <= cv[3][1]):
            
            dist = lambda p: sqrt((p[0]-cc[2][0])**2+(p[1]-cc[2][1])**2)
            if dist((x,y)) > 2*r: return (0)
            elif dist((x,y)) < r: return (sor)
            else: return (sor*(2*r-dist((x,y)))/r)
            
    return (0)


if SHOW_POTENTIAL: 
    # plot for control
    fig = plt.figure()
    ax = plt.axes(projection='3d')

    x = y = np.arange(0, l+10*scaling_factor, 5*scaling_factor)
    X, Y = np.meshgrid(x, y)
    zs = np.array([T0z(x,y,(hc0+hc1)/2) for x,y in zip(np.ravel(X), np.ravel(Y))])
    Z = zs.reshape(X.shape)

    ax.scatter(X, Y, Z)
    plt.title('z coordianate of impressed current vector potential')

In [5]:
N = 300

x_lin = np.linspace(cv[0][0], cv[1][0], N)
y_lin = np.linspace(cv[0][1], cv[3][1], N)
z_lin = np.linspace(hc0, hc1, int(2/(hc1-hc0)))
# z_lin = np.linspace(hc0-marg, hc1+marg, int(N*(hc1-hc0+2*marg)/(cv[1][0]-cv[0][0]+2*marg)))


values = np.array([T0z(x,y,z) for x in x_lin for y in y_lin for z in z_lin])
values = values.reshape(len(x_lin),len(y_lin),len(z_lin))
func = VoxelCoefficient((cv[0][0],cv[0][1],hc0), 
        (cv[1][0],cv[3][1],hc1), values, linear=True)

Draw(func, mesh)

NGSWebGuiWidget(value={'ngsolve_version': '6.2.2008-17-g2ebed44be', 'mesh_dim': 3, 'order2d': 2, 'order3d': 2,…

In [6]:
fes = HCurl(mesh, order = 2, dirichlet="outer", complex = True)#, nograds=True,)
u,v = fes.TnT()


# physical constants
mu0 = 4*np.pi*1e-7 
curdens = 2742/(r*(hc1-hc0)) # impressed current density in m (if scaling factor is 1e-3)
omega = 2*np.pi*50 # frequenzy
sigma = 3.526e7

# nu = CoefficientFunction([1/(mu0*1000) if mat in coil_mat else 1/mu0 for mat in mesh.GetMaterials()])

nu = CoefficientFunction(1/mu0)
# regularization term and conductivity term
kappa = CoefficientFunction([1j*omega*sigma if mat=='plate' else 1e-6*nu for mat in mesh.GetMaterials()])

a = BilinearForm(fes)
a += nu*curl(u)*curl(v)*dx + kappa*u*v*dx 
c = Preconditioner(a, "bddc")

f1 = LinearForm(fes)
f1 += CoefficientFunction(curdens)*source_norm*v*dx

f2 = LinearForm(fes)
f2 += CoefficientFunction(curdens)*func*CoefficientFunction((0,0,1))*curl(v)*dx

with TaskManager():
    a.Assemble()
    f1.Assemble()
    gfu1 = GridFunction(fes)

    f2.Assemble()
    gfu2 = GridFunction(fes)
    
    solvers.CG(sol=gfu1.vec, rhs=f1.vec, mat=a.mat, pre=c.mat, tol= 1e-8, printrates=False)
    solvers.CG(sol=gfu2.vec, rhs=f2.vec, mat=a.mat, pre=c.mat, tol= 1e-8, printrates=False)


B1 = lambda omega_t: (curl(gfu1)*exp(1j*omega_t)).real
B2 = lambda omega_t: (curl(gfu2)*exp(1j*omega_t)).real
      
Draw(B1(0), mesh, "B1", draw_surf=False)
Draw(B2(0), mesh, "B2", draw_surf=False)

NGSWebGuiWidget(value={'ngsolve_version': '6.2.2008-17-g2ebed44be', 'mesh_dim': 3, 'order2d': 2, 'order3d': 2,…

NGSWebGuiWidget(value={'ngsolve_version': '6.2.2008-17-g2ebed44be', 'mesh_dim': 3, 'order2d': 2, 'order3d': 2,…

In [34]:
%matplotlib
from matplotlib import pyplot as plt

xv = [0, 0.018, 0.036, 0.054, 0.072, 0.09, 0.108, 0.126, 0.144, 0.162,
         0.18, 0.198, 0.216, 0.234, 0.252, 0.27, 0.288]

val = {'0': np.array([B1(0)(mesh(x, 0.072, 0.034))[2] for x in xv]),
          '90': np.array([B1(np.pi/2)(mesh(x, 0.072, 0.034))[2] for x in xv])}
ref_val = {'0': np.array([-4.9, -17.88, -22.13,-20.19,-15.67,0.36,43.64,78.11,
                 71.55,60.44,53.91,51.62,53.81,56.91,59.24,52.78,27.61])*1e-4,
          '90': np.array([-1.16,2.84,4.15,4,3.07,2.31,1.89,4.97,12.61,14.15,13.04,
                         12.4,12.05,12.27,12.66,9.96,2.36])*1e-4}


plt.figure(0)
plt.plot(xv, val['0'], label='modeled')
plt.plot(xv, ref_val['0'], label='measured')
print("relative error", abs(np.divide(val['0']-ref_val['0'], ref_val['0'])))
plt.title('z-coord of B(0)')
plt.legend()

plt.figure(1)
plt.plot(xv, val['90'],label='modeled')
plt.plot(xv, ref_val['90'], label='measured')
print("relative error", abs(np.divide(val['90']-ref_val['90'], ref_val['90'])))
plt.title('z-coord of B(90)')
plt.legend()

# print("Version 1: {}, Version2: {}".format(B1(omega_t)(mip)[2], B2(omega_t)(mip)[2]))

Using matplotlib backend: Qt5Agg
relative error [1.11011677 0.18228577 0.0341745  0.1636058  0.26493066 6.77380682
 0.30267717 0.11577465 0.08629317 0.11062931 0.11755856 0.10424513
 0.1360692  0.12014039 0.14309344 0.3210129  0.4944248 ]
relative error [0.79475341 0.42093144 0.5105092  0.62032606 0.78462986 1.13995317
 1.98157397 0.52296976 0.2400299  0.08653006 0.20496491 0.17665021
 0.17143459 0.2258396  0.29042514 0.34405283 1.27753975]
